In [ ]:
# Copyright  2024 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Deepspeed with Ray on Vertex AI

* https://docs.ray.io/en/latest/train/examples/deepspeed/gptj_deepspeed_fine_tuning.html

### Configuration

In [ ]:
%pip install --user -q "google-cloud-aiplatform[ray]>=1.56.0" \
                        "ray[data,train,tune,serve]==2.33.0"

In [ ]:
import numpy as np
import pandas as pd
import os

import ray

In [ ]:
ray.__version__

In [ ]:
# @title Define constants
PROJECT_NBR = "721521243942"
PROJECT_ID = "ai-hangsik"
REGION="us-central1"
RAY_CLUSTER_NM = "ray33-cluster-20250218-085159"

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

### Connect to Ray on Vertex AI

In [ ]:
ray.shutdown()

In [ ]:
RAY_ADDRESS=f"vertex_ray://projects/{PROJECT_NBR}/locations/{REGION}/persistentResources/{RAY_CLUSTER_NM}"
print(f"RAY_ADDRESS:{RAY_ADDRESS}")

RUNTIME_ENV = {
  "pip": [
      "google-cloud-aiplatform[ray]>=1.56.0",
      "ray[data,train,tune,serve]==2.33.0",
      "datasets",
      "evaluate",
      "accelerate==0.18.0",
      "transformers==4.26.0",
      # "torch==1.12.0",
      
      "torch==2.0.0",  # for CUDA 11.8 : https://pytorch.org/get-started/previous-versions/
      "deepspeed==0.14.4", # https://github.com/huggingface/alignment-handbook/issues/180
      "numpy<2",
      
      "setuptools",
      "ipython",
      "scikit-learn",
      "ninja",
      "triton<=3.1.0"  # https://github.com/deepspeedai/DeepSpeed/issues/7028
      
  ],
}

# pytorch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1  pytorch-cuda=11.8

# runtime_env = {"env_vars": {"NCCL_SOCKET_IFNAME": "ens5"}}


### Connect to Ray on Vertex AI

In [ ]:
ray.init(address=RAY_ADDRESS,runtime_env=RUNTIME_ENV)

### Model and cluster configuration

In [ ]:
model_name = "EleutherAI/gpt-j-6B"
use_gpu = True
num_gpus = 2
num_workers = 3
cpus_per_worker = 6  # g2-standard-24 : https://cloud.google.com/compute/docs/gpus#l4-gpus

### Dataset

In [ ]:
import ray.data

@ray.remote
def data_manage():

    from datasets import load_dataset
    from datasets.dataset_dict import DatasetDict

    print("Loading tiny_shakespeare dataset")
    current_dataset = load_dataset("tatsu-lab/alpaca")

    slice_dataset = DatasetDict({'train': current_dataset['train'].select(range(2000))})

    train_dataset, validation_dataset= slice_dataset['train'].train_test_split(test_size=0.1).values()
    dataset = DatasetDict({'train': train_dataset, 'validation': validation_dataset})

    ray_datasets = {
        "train": ray.data.from_huggingface(dataset["train"]),
        "validation": ray.data.from_huggingface(dataset["validation"]),
    }

    return ray_datasets


In [ ]:
ray_datasets = ray.get(data_manage.remote())

### Training

In [ ]:
block_size = 512


In [ ]:
from transformers import AutoTokenizer

def split_text(batch: pd.DataFrame) -> pd.DataFrame:
    text = list(batch["text"])
    flat_text = "".join(text)
    split_text = [
        x.strip()
        for x in flat_text.split("\n")
        if x.strip() and not x.strip()[-1] == ":"
    ]
    return pd.DataFrame(split_text, columns=["text"])


def tokenize(batch: pd.DataFrame) -> dict:
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    tokenizer.pad_token = tokenizer.eos_token
    ret = tokenizer(
        list(batch["text"]),
        truncation=True,
        max_length=block_size,
        padding="max_length",
        return_tensors="np",
    )
    ret["labels"] = ret["input_ids"].copy()
    return dict(ret)


processed_datasets = {
    key: (
        ds.map_batches(split_text, batch_format="pandas")
        .map_batches(tokenize, batch_format="pandas")
    )
    for key, ds in ray_datasets.items()
}
processed_datasets


In [ ]:
import evaluate
import torch
from transformers import (
    Trainer,
    TrainingArguments,
    GPTJForCausalLM,
    AutoTokenizer,
    default_data_collator,
)
from transformers.utils.logging import disable_progress_bar, enable_progress_bar

from ray import train
from ray.train.huggingface.transformers import prepare_trainer, RayTrainReportCallback

def train_func(config):
    # Use the actual number of CPUs assigned by Ray
    os.environ["OMP_NUM_THREADS"] = str(
        train.get_context().get_trial_resources().bundles[-1].get("CPU", 1)
    )
    # Enable tf32 for better performance
    torch.backends.cuda.matmul.allow_tf32 = True

    batch_size = config.get("batch_size", 4)
    epochs = config.get("epochs", 2)
    warmup_steps = config.get("warmup_steps", 0)
    learning_rate = config.get("learning_rate", 0.00002)
    weight_decay = config.get("weight_decay", 0.01)
    steps_per_epoch = config.get("steps_per_epoch")

    deepspeed = {
        "fp16": {
            "enabled": "auto",
            "initial_scale_power": 8,
            "hysteresis": 4,
            "consecutive_hysteresis": True,
        },
        "bf16": {"enabled": "auto"},
        "optimizer": {
            "type": "AdamW",
            "params": {
                "lr": "auto",
                "betas": "auto",
                "eps": "auto",
            },
        },
        "zero_optimization": {
            "stage": 3,
            "offload_optimizer": {
                "device": "cpu",
                "pin_memory": True,
            },
            "overlap_comm": True,
            "contiguous_gradients": True,
            "reduce_bucket_size": "auto",
            "stage3_prefetch_bucket_size": "auto",
            "stage3_param_persistence_threshold": "auto",
            "gather_16bit_weights_on_model_save": True,
            "round_robin_gradients": True,
        },
        "gradient_accumulation_steps": "auto",
        "gradient_clipping": "auto",
        "steps_per_print": 10,
        "train_batch_size": "auto",
        "train_micro_batch_size_per_gpu": "auto",
        "wall_clock_breakdown": False,
    }

    print("Preparing training arguments")
    training_args = TrainingArguments(
        "output",
        logging_steps=1,
        save_strategy="steps",
        save_steps=steps_per_epoch,
        max_steps=steps_per_epoch * epochs,
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=1,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        warmup_steps=warmup_steps,
        label_names=["input_ids", "attention_mask"],
        push_to_hub=False,
        report_to="none",
        disable_tqdm=True,  # declutter the output a little
        fp16=True,
        gradient_checkpointing=True,
        deepspeed=deepspeed,
    )
    disable_progress_bar()

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token

    print("Loading model")

    model = GPTJForCausalLM.from_pretrained(model_name, use_cache=False)
    model.resize_token_embeddings(len(tokenizer))

    print("Model loaded")

    enable_progress_bar()

    metric = evaluate.load("accuracy")

    train_ds = train.get_dataset_shard("train")
    eval_ds = train.get_dataset_shard("validation")

    train_ds_iterable = train_ds.iter_torch_batches(
        batch_size=batch_size,
        local_shuffle_buffer_size=train.get_context().get_world_size() * batch_size,
    )
    eval_ds_iterable = eval_ds.iter_torch_batches(batch_size=batch_size)

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds_iterable,
        eval_dataset=eval_ds_iterable,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
        data_collator=default_data_collator,
    )

    # Add callback to report checkpoints to Ray Train
    trainer.add_callback(RayTrainReportCallback())
    trainer = prepare_trainer(trainer)
    trainer.train()

In [ ]:

storage_path = "gs://sllm_checkpoints/tmp_store/deepspeed/"


In [ ]:

batch_size = 16
# train_ds_size = processed_datasets["train"].count()
# steps_per_epoch = train_ds_size // (batch_size * num_workers)

train_ds_size = 1800    
steps_per_epoch = round(train_ds_size / (batch_size * num_workers))
steps_per_epoch

In [ ]:
from ray.train.torch import TorchTrainer
from ray.train import RunConfig, ScalingConfig

trainer = TorchTrainer(
    train_loop_per_worker=train_func,
    train_loop_config={
        "epochs": 2,
        "batch_size": batch_size,  # per device
        "steps_per_epoch": steps_per_epoch,
    },
    scaling_config=ScalingConfig(
        num_workers=num_workers,
        use_gpu=use_gpu,
        resources_per_worker={
            "GPU": num_gpus, 
            "CPU": cpus_per_worker
        },
    ),
    datasets=processed_datasets,
    run_config=RunConfig(storage_path=storage_path),
)

In [ ]:
results = trainer.fit()

In [ ]:
# checkpoint = results.checkpoint
# checkpoint

In [ ]:
# import os

# os.system(f"aws s3 sync s3://{checkpoint.path} /mnt/local_storage/")

In [ ]:
# from transformers import pipeline, AutoTokenizer, GPTJForCausalLM

# model = GPTJForCausalLM.from_pretrained("/mnt/local_storage/checkpoint")
# tokenizer = AutoTokenizer.from_pretrained("/mnt/local_storage/checkpoint")

# pipe = pipeline(
#     model=model,
#     tokenizer=tokenizer,
#     task="text-generation",
#     torch_dtype=torch.float16,
#     device_map="auto",
# )

In [ ]:
# # Generate from prompts!
# for sentence in pipe(
#     ["Romeo and Juliet", "Romeo", "Juliet"], do_sample=True, min_length=20
# ):
#     print(sentence)